In [1]:
import numpy as np
#Load the data
data = np.load('teapot.npy')
flattened = np.load('teapot_flat.npy')

sample_rate = 0.002

In [3]:
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(lowcut, highcut, fs, data, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

import functools


lowcut = 1
highcut = 10
fs =  1/sample_rate

flattened_low = np.apply_along_axis(functools.partial(butter_bandpass_filter, lowcut, highcut, fs), 2, flattened)

lowcut = 5
highcut = 60

flattened_mid = np.apply_along_axis(functools.partial(butter_bandpass_filter, lowcut, highcut, fs), 2, flattened)

lowcut = 40
highcut = 200

flattened_high = np.apply_along_axis(functools.partial(butter_bandpass_filter, lowcut, highcut, fs), 2, flattened)

In [4]:
vm_high = np.percentile(flattened_high, 99)
vm_mid = np.percentile(flattened_mid, 99)
vm_low = np.percentile(flattened_low, 99)

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from numpy.ma import masked_array
from matplotlib.colors import Normalize
from matplotlib.colors import ListedColormap
%matplotlib inline

def show_time_slice(i):
    print(i)
    cmaps_with_alpha = [] 
    for cmap in (cm.Reds,cm.Blues,cm.Greens):
        # Get the colormap colors
        my_cmap = cmap(np.arange(cmap.N))        
        top_half = my_cmap[int(cmap.N/2):,:]
        my_cmap = np.concatenate((top_half, np.flip(top_half, axis=0)), axis=0)
        
        # Set alpha
        my_cmap[:,-1] = np.concatenate((np.linspace(1, 0, cmap.N/2), np.linspace(0, 1, cmap.N/2)))

        cmaps_with_alpha.append(ListedColormap(my_cmap))

    cm_reds_alpha = cmaps_with_alpha[0]
    cm_blues_alpha = cmaps_with_alpha[1]
    cm_greens_alpha = cmaps_with_alpha[2] 

    colors_reds = cm_reds_alpha(Normalize(-vm_high, vm_high, clip=True)(flattened_high[:,:,i]))
    colors_greens = cm_greens_alpha(Normalize(-vm_mid, vm_mid, clip=True)(flattened_mid[:,:,i]))
    colors_blues = cm_blues_alpha(Normalize(-vm_low, vm_low, clip=True)(flattened_low[:,:,i]))

    plt.rcParams['figure.figsize'] = [16, 20]
    #plt.figure(figsize=(16, 20))
    fig,ax = plt.subplots()

    pa = ax.imshow(flattened_high[:,:,i], interpolation='nearest', cmap=cm_reds_alpha, aspect=0.5)
    cba = plt.colorbar(pa, shrink=0.5)
    
    pc = ax.imshow(flattened_mid[:,:,i], interpolation='nearest', cmap=cm_greens_alpha, aspect=0.5)
    cbc = plt.colorbar(pc, shrink=0.5)
    
    pb = ax.imshow(flattened_low[:,:,i], interpolation='nearest', cmap=cm_blues_alpha, aspect=0.5)
    cbb = plt.colorbar(pb, shrink=0.5)

    plt.xlabel('Crossline')
    plt.ylabel('Inline')
    cba.set_label('Highs')
    cbb.set_label('Lows')
    cbc.set_label('Mids')
    plt.show()

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(show_time_slice, i=widgets.IntSlider(min=0, max=flattened.shape[2], step=1, value=589))


interactive(children=(IntSlider(value=589, description='i', max=1501), Output()), _dom_classes=('widget-intera…

<function __main__.show_time_slice(i)>